In [2]:
import os
import csv
import json
import pandas as pd

In [3]:
import sys

sys.path.insert(0, "..")

* Procesamiento de los datos de navegación extraídos en estructura de carpetas y en formato .json

In [4]:
curr_date = pd.to_datetime('today').date().strftime("%d%m%Y")
ruta_principal = '../datos/marta/download_data/backup_' + curr_date +'/users' 


In [22]:
def procesar_json(ruta):
    # Crea una lista para almacenar los datos extraídos
    datos = []
    # Recorre cada archivo en la ruta proporcionada
    for archivo in os.listdir(ruta):
        
        # Si el archivo es un archivo JSON
        if archivo.endswith('.json'):
            # Abre el archivo JSON y extrae los datos
            with open(os.path.join(ruta, archivo)) as f:
                datos_json = json.load(f)
                # Agrega los datos a la lista de datos
                datos.append(datos_json)
                
    # Devuelve la lista de datos
    return datos


# Crea una lista para almacenar todos los datos de los archivos JSON
todos_los_datos = []

# Recorre cada subcarpeta en el cuarto nivel y procesa los archivos JSON
for ruta, carpetas, archivos in os.walk(ruta_principal):
    if ruta_principal in carpetas:
        carpetas.remove(ruta_principal)
    if len(os.path.relpath(ruta, ruta_principal).split(os.sep)) == 3: #dentro de 'users' hay 3 niveles de carpetas hasta llegar a los json
        datos = procesar_json(ruta)
        todos_los_datos.extend(datos)


# Crea un archivo CSV y escribe los datos en él
with open(r'../datos/marta/data_processed/datos' + curr_date + '.csv', 'w', newline='') as f:
    escritor_csv = csv.writer(f)
    # Escribe el encabezado del archivo CSV
    escritor_csv.writerow(['userId','sessionId', 'timestamp', 'navigation', 'eventType'])
    # Escribe los datos de cada fila en el archivo CSV
    for datos in todos_los_datos:
        escritor_csv.writerow([datos['userId'].get('value'), datos['sessionId'].get('value'), datos['timestamp'].get('value'), datos['navigation'].get('value'),datos.get('eventType', {}).get('value', '') ])


# Transformación de los datos

In [65]:
path_import = r'../datos/marta/data_processed/datos' + curr_date + '.csv'
#path_import = r'..\datos\marta\data_processed\datos13042023.csv'
df = pd.read_csv(path_import)
df

,userId,sessionId,timestamp,navigation,eventType
0,MGTOG86ByNRVU9ZjGpXPPz6GXu33,a39d00c0-e66e-11ed-b45e-5b898789a146,1682759870296,/noticias,NaN
1,MGTOG86ByNRVU9ZjGpXPPz6GXu33,a39d00c0-e66e-11ed-b45e-5b898789a146,1682759860499,/deportes/futbol/reservasFutbol,NaN
2,MGTOG86ByNRVU9ZjGpXPPz6GXu33,a39d00c0-e66e-11ed-b45e-5b898789a146,1682759847021,/carrito/pago/pagoCompletado,NaN
3,MGTOG86ByNRVU9ZjGpXPPz6GXu33,a39d00c0-e66e-11ed-b45e-5b898789a146,1682759857785,/pistaReservada,NaN
4,MGTOG86ByNRVU9ZjGpXPPz6GXu33,a39d00c0-e66e-11ed-b45e-5b898789a146,1682759864065,/deportes/futbol/reservasFutbol,NaN
...,...,...,...,...,...
128,1qyqHPsqRSUGTHWysepdiRUfcED3,33483f70-e66d-11ed-bccd-237e16841026,1682759253971,/deportes,THEME_SWITCH_DARK
129,1qyqHPsqRSUGTHWysepdiRUfcED3,33483f70-e66d-11ed-bccd-237e16841026,1682759304637,/deportes/padel,NaN
130,1qyqHPsqRSUGTHWysepdiRUfcED3,33483f70-e66d-11ed-bccd-237e16841026,1682759324740,/deportes/baloncesto/reservasBaloncesto,NaN
131,1qyqHPsqRSUGTHWysepdiRUfcED3,33483f70-e66d-11ed-bccd-237e16841026,1682759274972,/carrito,NaN


In [66]:
df = df.rename(columns={"sessionId": "visitId", "navigation": "pagePath"})
##df["visitNumber"] = df["visitId"]
df = df.sort_values(['userId','visitId','timestamp'], ascending=[True,True, True])
df = df[['userId','visitId','timestamp','pagePath','eventType']]

# Agregar columna de contador
df['visitNumber'] = df.groupby('userId').visitId.rank(method='dense').astype(int)

df = df[['userId','visitId','visitNumber','timestamp','pagePath','eventType']]

df["time"] = 0
grupos = df.groupby(["visitId"])

for nombre_grupo, grupo in grupos:
    primer_registro = grupo.iloc[0]["timestamp"]
    for i in range(len(grupo)):
        #diferencia = (grupo.iloc[i]["timestamp"] - primer_registro) / 1000  ## esto era para pasarlo a segundos
        diferencia = grupo.iloc[i]["timestamp"] - primer_registro
        df.at[grupo.index[i], "time"] = diferencia

df["hits"] = df.groupby("visitId")["visitId"].transform("count")
df["browser"] = 'Firefox'
df["deviceCategory"] = 'desktop'
# Obtener la longitud máxima de las rutas
max_levels = df['pagePath'].str.count('/').max()

# Crear columnas adicionales
new_columns = ['pagePathLevel{}'.format(i) for i in range(1, max_levels+1)]
df[new_columns] = df['pagePath'].str.strip('/').str.split('/', expand=True).fillna('')
cols = ["pagePathLevel1", "pagePathLevel2", "pagePathLevel3"]

# Crear una función que agregue barras a cada valor de la columna si la siguiente columna no está vacía
def add_slash(x):
    for i in range(len(x)-1):
        if x[i+1] != "":
            x[i] = "/" + x[i] + "/"
        else:
            x[i] = "/" + x[i]
    x[-1] = "/" + x[-1]
    return x

# Aplicar la función a las columnas seleccionadas
df[cols] = df[cols].fillna("").apply(add_slash, axis=1)

# Eliminar las barras en columnas que estaban originalmente vacías
df[cols] = df[cols].replace(r'^/$', '', regex=True)

#CREACIÓN CAMPO TYPE: diferenciar entre PAGE y EVENT
df['type'] = df['eventType'].apply(lambda x: 'PAGE' if pd.isnull(x) else 'EVENT')
#COLUMNAS DE EVENTOS
df['eventCategory'] = df['eventType'].apply(lambda x: '' if pd.isnull(x) else 'Promoting sports')
df['eventAction'] = '' #ACCION lanzada por el evento
# Asignar valores a la ACCION según la condición
df.loc[df['eventType'] == 'THEME_SWITCH_DARK', 'eventAction'] = 'Theme Switch Click'
df.loc[df['eventType'] == 'THEME_SWITCH_LIGHT', 'eventAction'] = 'Theme Switch Click'
#### CUANDO TENGA MAS EVENTOS MOFICAR LA ACCION CON EL VALOR CORRESPONDIENTE ####
df['eventValue'] = '' #este campo siempre está vacío
df = df.rename(columns={"eventType": "eventLabel"}) #ETIQUETA del evento

#TRANFORMACIÓN EN int DEL CAMPO visitNumber: Modificar si es necesario
""" df['visitNumber_new'] = df['visitNumber'].rank(method='dense').astype(int)
df = df.drop('visitNumber', axis=1)
df = df.rename(columns={"visitNumber_new": "visitNumber"}) """

#reordenamos columnas
df = df[['userId','visitId', 'visitNumber', 'time', 'hits', 'browser', 'deviceCategory', 'type', 'pagePath',  'pagePathLevel1', 'pagePathLevel2','pagePathLevel3','eventCategory','eventAction','eventLabel','eventValue' ]]
df.head()

,userId,visitId,visitNumber,time,hits,browser,deviceCategory,type,pagePath,pagePathLevel1,pagePathLevel2,pagePathLevel3,eventCategory,eventAction,eventLabel,eventValue
71,1qyqHPsqRSUGTHWysepdiRUfcED3,2a496ec0-e66e-11ed-8679-f37ca261cb04,1,0,16,Firefox,desktop,PAGE,/home,/home,,,,,NaN,
82,1qyqHPsqRSUGTHWysepdiRUfcED3,2a496ec0-e66e-11ed-8679-f37ca261cb04,1,2471,16,Firefox,desktop,PAGE,/carrito,/carrito,,,,,NaN,
74,1qyqHPsqRSUGTHWysepdiRUfcED3,2a496ec0-e66e-11ed-8679-f37ca261cb04,1,3719,16,Firefox,desktop,PAGE,/carrito/pago,/carrito/,/pago,,,,NaN,
78,1qyqHPsqRSUGTHWysepdiRUfcED3,2a496ec0-e66e-11ed-8679-f37ca261cb04,1,5322,16,Firefox,desktop,PAGE,/carrito/pago/pagoCompletado,/carrito/,/pago/,/pagoCompletado,,,NaN,
80,1qyqHPsqRSUGTHWysepdiRUfcED3,2a496ec0-e66e-11ed-8679-f37ca261cb04,1,6850,16,Firefox,desktop,PAGE,/home,/home,,,,,NaN,


In [67]:
df[df["type"]=='EVENT']

,userId,visitId,visitNumber,time,hits,browser,deviceCategory,type,pagePath,pagePathLevel1,pagePathLevel2,pagePathLevel3,eventCategory,eventAction,eventLabel,eventValue
112,1qyqHPsqRSUGTHWysepdiRUfcED3,33483f70-e66d-11ed-bccd-237e16841026,2,13756,26,Firefox,desktop,EVENT,/deportes,/deportes,,,Promoting sports,Theme Switch Click,THEME_SWITCH_LIGHT,
128,1qyqHPsqRSUGTHWysepdiRUfcED3,33483f70-e66d-11ed-bccd-237e16841026,2,33513,26,Firefox,desktop,EVENT,/deportes,/deportes,,,Promoting sports,Theme Switch Click,THEME_SWITCH_DARK,
103,1qyqHPsqRSUGTHWysepdiRUfcED3,4aa805f0-e66e-11ed-9f0e-cde1334bf084,3,10913,23,Firefox,desktop,EVENT,/home,/home,,,Promoting sports,Theme Switch Click,THEME_SWITCH_LIGHT,
38,MGTOG86ByNRVU9ZjGpXPPz6GXu33,8a4ee250-e66e-11ed-a5dc-c349a522dee1,1,10467,18,Firefox,desktop,EVENT,/pistaReservada,/pistaReservada,,,Promoting sports,Theme Switch Click,THEME_SWITCH_DARK,
26,MGTOG86ByNRVU9ZjGpXPPz6GXu33,8a4ee250-e66e-11ed-a5dc-c349a522dee1,1,14959,18,Firefox,desktop,EVENT,/noticias/eventos,/noticias/,/eventos,,Promoting sports,Theme Switch Click,THEME_SWITCH_LIGHT,
30,MGTOG86ByNRVU9ZjGpXPPz6GXu33,8a4ee250-e66e-11ed-a5dc-c349a522dee1,1,17525,18,Firefox,desktop,EVENT,/noticias/eventos,/noticias/,/eventos,,Promoting sports,Theme Switch Click,THEME_SWITCH_DARK,
31,MGTOG86ByNRVU9ZjGpXPPz6GXu33,8a4ee250-e66e-11ed-a5dc-c349a522dee1,1,24691,18,Firefox,desktop,EVENT,/deportes/padel/tiendaPadel,/deportes/,/padel/,/tiendaPadel,Promoting sports,Theme Switch Click,THEME_SWITCH_LIGHT,
10,MGTOG86ByNRVU9ZjGpXPPz6GXu33,a39d00c0-e66e-11ed-b45e-5b898789a146,2,2863,21,Firefox,desktop,EVENT,/home,/home,,,Promoting sports,Theme Switch Click,THEME_SWITCH_DARK,
7,MGTOG86ByNRVU9ZjGpXPPz6GXu33,a39d00c0-e66e-11ed-b45e-5b898789a146,2,17276,21,Firefox,desktop,EVENT,/deportes/tenis/reservasTenis,/deportes/,/tenis/,/reservasTenis,Promoting sports,Theme Switch Click,THEME_SWITCH_LIGHT,
8,MGTOG86ByNRVU9ZjGpXPPz6GXu33,a39d00c0-e66e-11ed-b45e-5b898789a146,2,21124,21,Firefox,desktop,EVENT,/pistaReservada,/pistaReservada,,,Promoting sports,Theme Switch Click,THEME_SWITCH_DARK,


In [68]:
df.columns

Index(['userId', 'visitId', 'visitNumber', 'time', 'hits', 'browser',
       'deviceCategory', 'type', 'pagePath', 'pagePathLevel1',
       'pagePathLevel2', 'pagePathLevel3', 'eventCategory', 'eventAction',
       'eventLabel', 'eventValue'],
      dtype='object')

In [69]:
### Creamos el maestro userId con sus diferentes visitId ####
df_maestro = df[['userId','visitId']].drop_duplicates()
df_maestro

,userId,visitId
71,1qyqHPsqRSUGTHWysepdiRUfcED3,2a496ec0-e66e-11ed-8679-f37ca261cb04
110,1qyqHPsqRSUGTHWysepdiRUfcED3,33483f70-e66d-11ed-bccd-237e16841026
96,1qyqHPsqRSUGTHWysepdiRUfcED3,4aa805f0-e66e-11ed-9f0e-cde1334bf084
28,MGTOG86ByNRVU9ZjGpXPPz6GXu33,8a4ee250-e66e-11ed-a5dc-c349a522dee1
20,MGTOG86ByNRVU9ZjGpXPPz6GXu33,a39d00c0-e66e-11ed-b45e-5b898789a146
46,dxqKsJ9Xl7SC5tHV8j1yW41KR7S2,0d7218b0-d6fb-11ed-b891-5330cfaeb505
64,dxqKsJ9Xl7SC5tHV8j1yW41KR7S2,f736df90-d6fa-11ed-8742-3f67350333b6


* Saving results

In [70]:
# Obtén la lista de valores únicos en la columna 'userId'
unique_user_ids = df['userId'].unique()

# Itera sobre cada 'userId' único y guarda un archivo CSV separado para cada uno
for user_id in unique_user_ids:
    # Filtra el dataframe para obtener solo los registros del usuario actual
    user_df = df[df['userId'] == user_id]
    
    # Guarda el dataframe separado como un archivo CSV con un nombre único basado en el 'userId'
    filename = f'Nivel-user_{user_id}.csv'
    user_df[['visitId', 'visitNumber', 'time', 'hits', 'browser',
       'deviceCategory', 'type', 'pagePath', 'pagePathLevel1',
       'pagePathLevel2', 'pagePathLevel3', 'eventCategory', 'eventAction',
       'eventLabel', 'eventValue']].to_csv('../datos/marta/data_processed/model/' + filename, index=False)

In [71]:
""" df.to_csv(r'..\datos\marta\data_processed\model\1-Nivel-' + curr_date + '.csv', sep=',', index=False)

###### NO ENTIENDO QUÉ DIFERENCIA TIENE QUE HABER ENTRE TODOS ESTOS EXCELS ######
df.to_csv(r'..\datos\marta\data_processed\model\2-Nivel-' + curr_date + '.csv', sep=',', index=False)
df.to_csv(r'..\datos\marta\data_processed\model\3-Nivel-' + curr_date + '.csv', sep=',', index=False)
df.to_csv(r'..\datos\marta\data_processed\model\4-Nivel-' + curr_date + '.csv', sep=',', index=False)
df.to_csv(r'..\datos\marta\data_processed\model\5-Nivel-' + curr_date + '.csv', sep=',', index=False) """

" df.to_csv(r'..\\datos\\marta\\data_processed\\model\x01-Nivel-' + curr_date + '.csv', sep=',', index=False)\n\n###### NO ENTIENDO QUÉ DIFERENCIA TIENE QUE HABER ENTRE TODOS ESTOS EXCELS ######\ndf.to_csv(r'..\\datos\\marta\\data_processed\\model\x02-Nivel-' + curr_date + '.csv', sep=',', index=False)\ndf.to_csv(r'..\\datos\\marta\\data_processed\\model\x03-Nivel-' + curr_date + '.csv', sep=',', index=False)\ndf.to_csv(r'..\\datos\\marta\\data_processed\\model\x04-Nivel-' + curr_date + '.csv', sep=',', index=False)\ndf.to_csv(r'..\\datos\\marta\\data_processed\\model\x05-Nivel-' + curr_date + '.csv', sep=',', index=False) "